# TP - Computação Natural
#### "Predict whether a mammogram mass is benign or malignant"

1. BI-RADS assessment: 1 to 5 (ordinal)  
2. Age: patient's age in years (integer)
3. Shape: mass shape: round=1 oval=2 lobular=3 irregular=4 (nominal)
4. Margin: mass margin: circumscribed=1 microlobulated=2 obscured=3 ill-defined=4 spiculated=5 (nominal)
5. Density: mass density high=1 iso=2 low=3 fat-containing=4 (ordinal)
6. Severity: benign=0 or malignant=1 (binominal)

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from scipy import stats

## Get the Data

In [ ]:
data = pd.read_csv('mammographic_masses.data.txt')
data

** Convert missing data (indicated by a ?) into NaN and add the appropriate column names (BI_RADS, age, shape, margin, density, and severity) **

In [ ]:
data = data.replace('?',np.nan)
data.columns = ['BI_RADS','Age','Shape','Margin','Density','Severity']
data

** Drop BI_RADS column because it has no influence on the severity forecast **

In [ ]:
data = data.drop(columns=['BI_RADS'])

** Convert datatype 'object' to 'float64' **  

In [ ]:
data.info()

In [ ]:
data = data.astype(float)
data

In [ ]:
data.info()

In [ ]:
data.describe()

** Check missing values **

In [ ]:
print(data.isnull().sum(axis=0))
sns.heatmap(data.isnull(),yticklabels=False,cbar=False,cmap='viridis')

** The missing data seems randomly distributed, so we drop rows with missing data **

In [ ]:
data = data.dropna()
data.index = np.arange(1, len(data) + 1)
data

** !!! (Ou podemos meter os dados que faltavam, ver isto) !!! **

In [ ]:
data.describe()

In [ ]:
print(data.isnull().sum(axis=0))
sns.heatmap(data.isnull(),yticklabels=False,cbar=False,cmap='viridis')

## Exploratory Data Analysis

https://towardsdatascience.com/a-starter-pack-to-exploratory-data-analysis-with-python-pandas-seaborn-and-scikit-learn-a77889485baf#249d

https://towardsdatascience.com/comprehensive-guide-to-exploratory-data-analysis-of-habermans-survival-data-set-b33f0373c83a

** Auxiliar functions & General definitions **

In [ ]:
c_palette = ['tab:green','tab:red']

def categorical_summarized(dataframe, x=None, y=None, hue=None, palette='Set1', verbose=True):
    if x == None:
        column_interested = y
    else:
        column_interested = x
    series = dataframe[column_interested]
    print(series.describe())
    print('mode: ', series.mode())
    if verbose:
        print('='*80)
        print(series.value_counts())

    sns.countplot(x=x, y=y, hue=hue, data=dataframe, palette=palette)
    plt.show()

def quantitative_summarized(dataframe, x=None, y=None, hue=None, palette='Set1', ax=None, verbose=True, swarm=False):
    series = dataframe[y]
    print(series.describe())
    print('mode: ', series.mode())
    if verbose:
        print('='*80)
        print(series.value_counts())

    sns.boxplot(x=x, y=y, hue=hue, data=dataframe, palette=palette, ax=ax)

    if swarm:
        sns.swarmplot(x=x, y=y, hue=hue, data=dataframe,
                      palette=palette, ax=ax)

    plt.show()

** Countplot of the Severity (Benign 0 vs Malignant 1) **

In [ ]:
sns.set_style('whitegrid')
ax = sns.countplot(x='Severity',data=data,palette=c_palette)


total = len(data['Severity'])

for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:.1f}%'.format(100 * height/total),
            ha="center") 

** Severity on Age **

In [ ]:
quantitative_summarized(dataframe= data, y = 'Age', x = 'Severity', palette=c_palette, verbose=False, swarm=True)

In [ ]:
sns.set_style('whitegrid')
sns.FacetGrid(data, hue='Severity', size=8, palette=c_palette) \
    .map(sns.distplot, 'Age', bins=10) \
    .add_legend()
plt.xticks([0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100])
plt.show()

In [ ]:
sns.set_style('darkgrid')
g = sns.FacetGrid(data,hue='Severity',palette=c_palette,size=6,aspect=2)
g = g.map(plt.hist,'Age',bins=20,alpha=0.7).add_legend()
plt.xticks([15,18,22,25,28,32,35,38,42,45,48,52,55,59,62,65,69,72,75,79,82,86,89,93,96,100])
plt.show()

In [ ]:
sns.jointplot(x='Age', y='Severity', data=data, kind='kde')
plt.show()

** Severity on Shape (mass shape: round=1 oval=2 lobular=3 irregular=4) **

In [ ]:
categorical_summarized(data, y = 'Shape', hue='Severity', palette=c_palette)

In [ ]:
sns.violinplot(data = data, x='Severity', y='Shape', palette=c_palette)
sns.swarmplot(data = data, x='Severity', y='Shape', color = 'k', alpha = 0.6, palette=c_palette)

** Severity on Margin (mass shape: round=1 oval=2 lobular=3 irregular=4) **

In [ ]:
categorical_summarized(data, y = 'Margin', hue='Severity', palette=c_palette)

In [ ]:
sns.violinplot(data = data, x='Severity', y='Margin', palette=c_palette)
sns.swarmplot(data = data, x='Severity', y='Margin', color = 'k', alpha = 0.6, palette=c_palette)

** Severity on Density (mass density high=1 iso=2 low=3 fat-containing=4) **

In [ ]:
categorical_summarized(data, y = 'Density', hue='Severity', palette=c_palette)

In [ ]:
sns.violinplot(data = data, x='Severity', y='Density', palette=c_palette)
sns.swarmplot(data = data, x='Severity', y='Density', color = 'k', alpha = 0.6, palette=c_palette)

##### Detect outliers: https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba

### Detect Outliers using Box plot (Uni-variate outlier)

In [ ]:
sns.boxplot(x=data['Age'])

In [ ]:
sns.boxplot(x=data['Shape'])

In [ ]:
sns.boxplot(x=data['Margin'])

In [ ]:
sns.boxplot(x=data['Density'])

### Detect Outliers using Scatter plot (Multi-variate outlier)

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax.scatter(data['Age'], data['Shape'])
ax.set_xlabel('Age')
ax.set_ylabel('Shape')
#ax.set_ylabel('Margin')
#ax.set_ylabel('Density')
plt.show()

### Detect outliers using mathematical function Z-Score

In [ ]:
z = np.abs(stats.zscore(data))
threshold = 3
print(np.where(z > threshold))
# The first array contains the list of row numbers and second array respective column numbers

Column 3 (density) has all outliers

### Detect outliers using IQR Score
Similar to Z-Score

In [ ]:
Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
iqr = Q3 - Q1
print(iqr)

In [ ]:
# Não curti ...
print(data < (Q1 - 1.5 * iqr)) |(data > (Q3 + 1.5 * iqr))

## Data Preparation

### Remove Outliers using Z-Score

##### + explanations: https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-data-frame

In [ ]:
# Só fazer 1 vez
data = data[(np.abs(stats.zscore(data)) < 3).all(axis=1)]
data.index = np.arange(1, len(data) + 1)
data

### Converting pandas dataframes to numpy arrays

In [ ]:
X = data.drop('Severity',axis=1).to_numpy()
y = data['Severity'].to_numpy()
X

### Normalizing the attribute data using StandardScaler

** Fit scaler to the features **

In [ ]:
# Só fazer 1 vez
scaler = StandardScaler()
scaler.fit(X)
scaler.mean_

** Transform the features to a scaled version **

In [ ]:
X = scaler.transform(X)
X

## Train Test Split

** Create training and testing sets of the data **

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train

## Neural Networks

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

def buildModel(hidden_layers, nodes_per_layer, activation_fn, optimizer, loss_fn, metrics, inputs=4,):
    model = Sequential()
    #add input layer
    model.add(Dense(inputs, activation=activation_fn, input_shape=(inputs,)))

    #add hidden layers    
    for i in range(hidden_layers):
        model.add(Dense(nodes_per_layer, activation=activation_fn))

    #add output layer
    model.add(Dense(1,activation=activation_fn))

    #compile model
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=metrics)

    return model

In [ ]:
testModel = buildModel(hidden_layers=2, nodes_per_layer=3, activation_fn='relu',  optimizer='rmsprop', loss_fn='binary_crossentropy', metrics=['accuracy'])

history = testModel.fit(X_train, y_train,
          epochs=20,
          batch_size=128, validation_split=0.2)

score = testModel.evaluate(X_train, y_train, batch_size=128, verbose=1)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

# Visualize history
# Plot history: Loss
plt.plot(history.history['loss'])
plt.title('Validation loss history')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.show()

# Plot history: Accuracy
plt.plot(history.history['accuracy'])
plt.title('Validation accuracy history')
plt.ylabel('Accuracy value (%)')
plt.xlabel('No. epoch')
plt.show()